In [ ]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

import import_ipynb
from constants import NUM_SPEAKER_PER_BATCH, NUM_UTT_PER_SPEAKER

In [ ]:
def simMat_loss(y_true, y_pred):

  def normalize(x): return x/tf.sqrt(tf.reduce_sum(x**2, axis=-1, keepdims=True)+1e-6)

  N = NUM_SPEAKER_PER_BATCH  # 화자 수
  M = NUM_UTT_PER_SPEAKER   # 화자당 발화 수

  embedded_split = tf.reshape(normalize(y_pred), shape=[N, M, -1])  # [N][M][P]

  center = normalize(tf.reduce_mean(embedded_split, axis=1))  # 화자마다 모든 발화의 평균. [N][P]
  center_except = normalize(tf.reshape(tf.reduce_sum(embedded_split, axis=1, keepdims=True)- embedded_split, shape=[N*M,-1]))  # 각 발화에 대해, 그 발화를 제외한 화자 내의 평균. [N][M][P]
  
  S = tf.concat(
      [tf.concat([tf.reduce_sum(center_except[i*M:(i+1)*M,:]*embedded_split[j,:,:], axis=1, keepdims=True) if i==j
                  else tf.reduce_sum(center[i:(i+1),:]*embedded_split[j,:,:], axis=1, keepdims=True) for i in range(N)],
                  axis=1) for j in range(N)], axis=0)

  S_correct = tf.concat([S[i*M:(i+1)*M, i:(i+1)] for i in range(N)], axis=0)
  total = -tf.reduce_sum(S_correct-tf.math.log(tf.reduce_sum(tf.exp(S), axis=1, keepdims=True) + 1e-6))

  return total

In [ ]:
def get_loss(loss_name):
    if loss_name=='cross_entropy': return keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    elif loss_name=='simMat_loss': return simMat_loss